In [3]:
from sparktools.sparkhandler import SparkHandler
from dataproc.dataproctools import get_extracted_wet, save_rdd, load_rdd
from pyspark import StorageLevel

We use the spark handler class so we can use a unified spark session across our functions. This essentially just helps us initialize the spark session and stores the session and context within itself

In [4]:
handler = SparkHandler(available_cores=4) # look at the parameters for SparkHandler to increase max memory
ss = handler.get_spark_session()
sc = handler.get_spark_context()

Here we can load in our str data. Do note, `get_extracted_wet` doesn't return the sample size exactly, but an approximate amount based on the wet we use

In [5]:
rawStrData = get_extracted_wet(spark_context=sc, approx_sample_size=10000, wet_paths_file="../data/wet.paths")

Typically, processing this code, especially at large scales, takes a lot of time. In order to not recompute every time we run this, we can save this RDD and load it later.

In [11]:
save_rdd(rawStrData, "../saved_intermediates/rawStrRDD", overwrite=True) # overwrite is false by default (NOTE: the ../ basically means to go back a directory. this ensures we create the folder in the root directory instead of the examples directory)

Loading the data is pretty straightforward with the load_rdd function

In [12]:
loaded_str_data = load_rdd(spark_context=sc, path_to_load="../saved_intermediates/rawStrRDD")

Here's what the start of a WET file looks like

In [13]:
loaded_str_data.persist(StorageLevel.MEMORY_AND_DISK) # We use persist here because I've often run into OOM errors at large data scales after triggering exection without it
firstRDDSnippet = loaded_str_data.take(1)[0][:10000] # We take the first 10000 chars of text
loaded_str_data.unpersist() # Unpersist the data to free up storage and memory
firstRDDSnippet

'WARC/1.0\r\nWARC-Type: warcinfo\r\nWARC-Date: 2025-11-21T04:54:24Z\r\nWARC-Filename: CC-MAIN-20251115201147-20251115231147-00569.warc.wet.gz\r\nWARC-Record-ID: <urn:uuid:27966d2c-0e0c-4bd3-ab9c-2e67f0e6f7eb>\r\nContent-Type: application/warc-fields\r\nContent-Length: 372\r\n\r\nSoftware-Info: ia-web-commons.3.0.1-SNAPSHOT-20251103031243\r\nExtracted-Date: Fri, 21 Nov 2025 04:54:24 GMT\r\nrobots: checked via crawler-commons 1.6-SNAPSHOT (https://github.com/crawler-commons/crawler-commons)\r\nisPartOf: CC-MAIN-2025-47\r\noperator: Common Crawl Admin (info@commoncrawl.org)\r\ndescription: Wide crawl of the web for November 2025\r\npublisher: Common Crawl\r\n\r\n\r\n\r\nWARC/1.0\r\nWARC-Type: conversion\r\nWARC-Target-URI: http://022hezi.com/tag/%E4%B9%9D%E5%8D%81%E5%85%AB\r\nWARC-Date: 2025-11-15T21:55:56Z\r\nWARC-Record-ID: <urn:uuid:02be2c7b-d20b-4e22-8a93-401bf6614637>\r\nWARC-Refers-To: <urn:uuid:a192054e-7280-436a-a2f9-919f3c2558e3>\r\nWARC-Block-Digest: sha1:NZ7VKS26DB7EWWQRGSA3B2G

And this is what it looks like when printed in a prettier format

In [14]:
print(firstRDDSnippet)

WARC/1.0
WARC-Type: warcinfo
WARC-Date: 2025-11-21T04:54:24Z
WARC-Filename: CC-MAIN-20251115201147-20251115231147-00569.warc.wet.gz
WARC-Record-ID: <urn:uuid:27966d2c-0e0c-4bd3-ab9c-2e67f0e6f7eb>
Content-Type: application/warc-fields
Content-Length: 372

Software-Info: ia-web-commons.3.0.1-SNAPSHOT-20251103031243
Extracted-Date: Fri, 21 Nov 2025 04:54:24 GMT
robots: checked via crawler-commons 1.6-SNAPSHOT (https://github.com/crawler-commons/crawler-commons)
isPartOf: CC-MAIN-2025-47
operator: Common Crawl Admin (info@commoncrawl.org)
description: Wide crawl of the web for November 2025
publisher: Common Crawl



WARC/1.0
WARC-Type: conversion
WARC-Target-URI: http://022hezi.com/tag/%E4%B9%9D%E5%8D%81%E5%85%AB
WARC-Date: 2025-11-15T21:55:56Z
WARC-Record-ID: <urn:uuid:02be2c7b-d20b-4e22-8a93-401bf6614637>
WARC-Refers-To: <urn:uuid:a192054e-7280-436a-a2f9-919f3c2558e3>
WARC-Block-Digest: sha1:NZ7VKS26DB7EWWQRGSA3B2GJXBHDFZWZ
WARC-Identified-Content-Language: zho
Content-Type: text/plain


This is just to show what the types of data look like

In [15]:
type(firstRDDSnippet), type(loaded_str_data)

(str, pyspark.core.rdd.RDD)

### TODO for Raamesh:
separate out the metadata from the raw text into key-value pairs and filter the data for english. final output should be key: domain, and value: raw text